# Broadcast

### But : Faire un broadcast, départ décalé 


#### Informations générales

In [71]:
experiment_name = "Time_Bcast"

In [72]:
%%bash -s
git log -n 1
python --version

commit c928dda523096b5637d95b56790991666749649c
Author: Najwa Ez Zine <nezzine@frennes>
Date:   Wed Jun 13 15:51:39 2018 +0200

    Ressources update


Python 2.7.13


#### Préparation de l'environnement

In [73]:
%%bash -s
pip3 install --user execo
pip3 install --user requests

  Using cached https://files.pythonhosted.org/packages/cc/15/e1c318dbc20032ffbe5628837ca0de2d5b116ffd1b849c699634010f6a5d/requests-2.19.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bd/c9/6fdd990019071a4a32a5e7cb78a1d92c53851ef4f56f62a3486e6a7d8ffb/urllib3-1.23-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4bb1ddec7ca55ec7510b22e4c51f14098443b8/chardet-3.0.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/4b/2a/0276479a4b3caeb8a8c1af2f8e4355746a97fab05a372e4a2c6a6b876165/idna-2.7-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7c/e6/92ad559b7192d846975fc916b65f667c7b8c3a32bea7372340bfe9a15fa5/certifi-2018.4.16-py2.py3-none-any.whl


In [74]:
import execo
from execo import *
from execo_g5k import *
from execo_engine import *
import os
import sys
import json
import time
import datetime
import re
from shutil import copy
from subprocess import check_output
import random
from threading import Thread
import math
import collections
from collections import deque

##### Réservation noeuds

In [75]:
# Nom du job
jobname = ' '
# Nombre de noeuds
nodecount = 3
# Temps réservation
walltime = "3:0:0" 

# filters out Nantes's econome cluster
resources_selection = "-p \"cluster in ('ecotype','parasilo','grisou')\"" 

# Nancy
site = "rennes"

In [76]:
jobs = get_current_oar_jobs()
jobid = None
waiting_jobs = []
while jobs:
    j, site = jobs.pop()
    info = get_oar_job_info(j, site)
    if info['name'] == jobname:
        if info['state'] == 'Running':
            jobid = j
            print("A {} job is already running, using it. jobid is {}".format(jobname, jobid))
            break
        else:
            waiting_jobs.append(j)
if not jobid and not waiting_jobs:
    jobspec = OarSubmission(resources="/cluster=1/nodes={}".format(nodecount), walltime=walltime,
                            additional_options=resources_selection, job_type="deploy", name=jobname)
    jobid, _ = oarsub([(jobspec, site)]).pop()
    print("New job submitted, jobid is {}".format(jobid))
elif not jobid:
    print("One or more {} jobs exist ({}) but are not running.\n"
          " Connect to the frontend to see what is happening, and/or run the cell again.".format(
          jobname, ", ".join([str(j) for j in waiting_jobs])))

New job submitted, jobid is 1015986


On les range

In [77]:
nodes = get_oar_job_nodes(jobid)
nodes.sort(key=lambda n: n.address)
nodes

[Host('parasilo-28.rennes.grid5000.fr'),
 Host('parasilo-3.rennes.grid5000.fr'),
 Host('parasilo-4.rennes.grid5000.fr')]

##### Déploiement de l'environnement

In [78]:
force_redeploy = False # set to True to force redeploying the OS on the nodes in the deployment section
environment_dsc_file = 'src/debian9-x64-bigdata-tutorial.yaml' # filename of the kadeploy environment file (YAML)

In [79]:
deployment = Deployment(hosts=nodes, env_file = os.path.abspath(environment_dsc_file),
                        other_options="-r ext4 --no-debug-mode")

deploy_ok, deploy_failed = deploy(deployment, check_deployed_command=not force_redeploy,
                              stdout_handlers=[sys.stdout],
                              stderr_handlers=[sys.stderr])

rennes: Deployment #D-68d315a1-cc2e-41f3-92fe-39cd07806b64 started
rennes: Grab the key file /home/nezzine/.ssh/authorized_keys
rennes: Grab the tarball file http://public.nancy.grid5000.fr/~pneyron/debian9-x64-bigdata-tutorial.tar.gz
rennes: Grab the postinstall file server:///grid5000/postinstalls/g5k-postinstall.tgz
rennes: Launching a deployment on parasilo-[3-4,28].rennes.grid5000.fr
rennes: Performing a Deploy[SetDeploymentEnvUntrusted] step
rennes:   switch_pxe
rennes:   reboot
rennes:    * Performing a soft reboot on parasilo-[3-4,28].rennes.grid5000.fr
rennes:   wait_reboot
rennes:   send_key_in_deploy_env
rennes:   create_partition_table
rennes:   format_deploy_part
rennes:   mount_deploy_part
rennes:   format_tmp_part
rennes:   format_swap_part
rennes: End of step Deploy[SetDeploymentEnvUntrusted] after 202s
rennes: Performing a Deploy[BroadcastEnvKascade] step
rennes:   send_environment
rennes:    * Broadcast time: 70s
rennes:   manage_admin_post_install
rennes:   manage_us

rennes: Warning: Permanently added 'rennes.grid5000.fr,172.16.111.106' (ECDSA) to the list of known hosts.
rennes: Connection to rennes.grid5000.fr closed.


#### Test : commande simple 'ls'

In [80]:
Remote_test = execo.action.Remote(cmd='ls',hosts=nodes,connection_params=None, process_args=None)
Remote_test.run().ok

True

#### Installation environnement sur noeud

In [81]:
Remote_install = execo.action.Remote(cmd='apt-get install g++ libboost-all-dev && wget http://gforge.inria.fr/frs/download.php/latestfile/8/SimGrid-3.18.tar.gz && tar -xvf SimGrid-3.18.tar.gz && cd SimGrid-3.18 && cmake -DCMAKE_INSTALL_PREFIX=/usr/local -Denable_smpi=on -Denable_documentation=off && make -j && make check && make install -j',hosts=nodes,connection_params={'user':'root'})

In [82]:
Remote_install.run().ok

True

#### Version Simgrid

In [83]:
execo.action.Remote(cmd='touch smpi_version.txt && smpicc --version > smpi_version.txt'
                    , hosts=nodes, connection_params={'user': 'root'
                    }).run()
version = ''
for i in range(0, nodecount):
    execo.action.Get(hosts=nodes[i], remote_files=['~/smpi_version.txt'
                     ], local_location='./smpicc_version-'
                     + get_host_shortname(nodes[i]) + '.txt',
                     connection_params={'user': 'root'}).run()
    version = version + execo.Process(cmd='head -n 1 ./smpicc_version-'
            + get_host_shortname(nodes[i]) + '.txt'
            ).run().stdout.replace('\n', '  ')
    execo.Process(cmd='rm ./smpicc_version-'
                  + get_host_shortname(nodes[i]) + '.txt').run()
version


'SimGrid version 3.18  SimGrid version 3.18  SimGrid version 3.18  '

In [84]:
path_base = "~/Stage-POLARIS/SimGrid/examples/SMPI/"+experiment_name
path_src = path_base + '/src/'
path_rslt = path_base + '/results/'

#### Récupération fichiers sources

In [ ]:
Src_get = execo.action.Remote(cmd='mkdir -p experiments_src',hosts=nodes,connection_params={'user':'root'})
Src_get.run().ok

In [ ]:
Remote_smpi_env = execo.action.Put(hosts=nodes,local_files=[path_src+"griffon.xml",path_src+"griffon_hostfile.txt",path_src+"broadcast.c",path_src+"Makefile"],remote_location='~/experiments_src',connection_params={'user':'root'})
Remote_smpi_env.run().ok

#### Execution de l'expérience

In [ ]:
Remote_smpi = execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS=2 MIN=0 MAX=10 N=2 DELAY=\"0 0\"',hosts=nodes,connection_params={'user':'root'})
Remote_smpi.run().ok

#### Récupération des résultats

In [ ]:
Result_get = \
    execo.Process(cmd='mkdir -p /home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/'
                   + experiment_name + '/results/'
                  + datetime.datetime.now().strftime('%Y-%m-%d'))
Result_get.run().ok

In [ ]:
for i in range(0, nodecount):
    execo.action.Get(hosts = nodes[i],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location = path_rslt
                     + datetime.datetime.now().strftime('%Y-%m-%d')
                     + '/' + get_host_shortname(nodes[i]) + '_'
                     + datetime.datetime.now().strftime('%Y-%m-%d_%H:%M'
                     ) + '.txt', connection_params={'user': 'root'
                     }).run().ok

#### Delayer

In [14]:
def init_tab(num, elt) :
    D = [elt] * num
    return D

In [15]:
# Parsing the results
def parser(num_node,date_precise):
    with open('/home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/Time_Bcast/results/'
               + date_precise.split('_')[0] + '/'
              + get_host_shortname(nodes[num_node]) + '_'
              + date_precise
              + '.txt') as f:
        return f.readlines()

In [16]:
# Converting and organizing data
def tab_conv(l,NPROCS) :
    S = init_tab(NPROCS, None)
        
    # Taking useful data    
    for i in range(0,NPROCS) :
        line = l[i].split()
        S[int(line[1])] = int(float(line[5]))
    return S

In [42]:
def delayer(num_node,NPROCS,date):
    D = init_tab(NPROCS,0)
    
    raw_data = parser(num_node,date)
    
    S = tab_conv(raw_data,NPROCS)
    
    for p in range(0, NPROCS):
        D[p] = S[p] - min(S)
    return [D,min(S)]

In [ ]:
# Example
delayer(0,10,'2018-06-04_12:38')

##### Shortcut : séquentiel

In [18]:
def setup() :
    Src_dir = execo.action.Remote(cmd='mkdir -p experiments_src',
                              hosts=nodes,
                              connection_params={'user': 'root'})
    Src_get = execo.action.Put(hosts=nodes, local_files=[path_src
                                   + 'griffon.xml', path_src
                                   + 'griffon_hostfile.txt', path_src
                                   + 'broadcast.c', path_src + 'Makefile'],
                                   remote_location='~/experiments_src',
                                   connection_params={'user': 'root'})
    Rslt_dir = \
        execo.Process(cmd='mkdir -p /home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/'
                   + experiment_name + '/results/'
                  + datetime.datetime.now().strftime('%Y-%m-%d'))

# On récup les src
    Src_dir.run().ok
    Src_get.run().ok
    Rslt_dir.run().ok


In [19]:

def parl_exec () :
    setup()
    # Nbr pcs
    NPROCS = 10
    
    # Nbr itérations
    N = 100
    
    D = init_tab(NPROCS,0)
    delay = ' '.join(str(e) for e in D)
    date = datetime.datetime.now().strftime('%Y-%m-%d');
    date_precise = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
    print(date_precise)
    
    # TODO //
    for i in range (0,nodecount) :
        MIN = (int)((i*N)/len(nodes))
        MAX = (int)(((i+1)*N)/len(nodes))
        print(get_host_shortname(nodes[i])+" => "+str(i)+" NPROCS = "+str(NPROCS)
             +" MIN = "+str(MIN)
             +" MAX = "+str(MAX)
             +" N = "+str(N)
             +" DELAY = '"+ delay +"'"
            )
        bool = execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS='
                                  +str(NPROCS)+' MIN='+str(MIN) 
                                  +' MAX='+str(MAX)+' N='+str(N)
                                  +" DELAY='"+delay
                                  +"'",hosts=nodes[i],connection_params={'user':'root'}).run().ok
        print(bool)
        execo.action.Get(hosts=nodes[i],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location= path_rslt
                     + date
                     + '/' + get_host_shortname(nodes[i]) + '_'
                     + date_precise + '.txt', connection_params={'user': 'root'
                     }).run().ok
        print(delayer(i,NPROCS,date_precise))
    return;

parl_exec()

for i in range(0, nodecount):
    execo.action.Get(hosts=nodes[i],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location=path_rslt
                     + datetime.datetime.now().strftime('%Y-%m-%d')
                     + '/' + get_host_shortname(nodes[i]) + '_'
                     + datetime.datetime.now().strftime('%Y-%m-%d_%H:%M'
                     ) + '.txt', connection_params={'user': 'root'
                     }).run().ok

2018-06-13_10:55
parasilo-5 => 0 NPROCS = 10 MIN = 0 MAX = 33 N = 100 DELAY = '0 0 0 0 0 0 0 0 0 0'
True
[2, 8, 1, 6, 8, 0, 1, 5, 6, 2]
parasilo-6 => 1 NPROCS = 10 MIN = 33 MAX = 66 N = 100 DELAY = '0 0 0 0 0 0 0 0 0 0'
True
[3, 0, 3, 6, 4, 5, 3, 4, 7, 4]
parasilo-7 => 2 NPROCS = 10 MIN = 66 MAX = 100 N = 100 DELAY = '0 0 0 0 0 0 0 0 0 0'
True
[6, 5, 2, 9, 4, 3, 2, 5, 0, 0]


#### Parallel simulation

In [20]:
# Thread pour la simulation // opt.
class ParSimer(Thread):

    """Thread chargé simplement d'afficher une lettre dans la console."""
    
    # Paramètrage du thread
    def __init__(self,i,MIN,MAX,N,DELAY):
        Thread.__init__(self)
        self.MIN = MIN
        self.MAX = MAX
        self.N = N
        self.DELAY = DELAY
        self.i = i
    
    # Set la valeur du délai
    def set_delay(self,delay):
        self.DELAY = delay
    
    def get_delay(self):
        return self.DELAY
    def get_MIN(self):
        return self.MIN
    def run(self):
        """Code à exécuter pendant l'exécution du thread."""
        """print(get_host_shortname(nodes[self.i])+" => "+str(self.i)+" NPROCS = "+str(NPROCS)
             +" MIN = "+str(self.MIN)
             +" MAX = "+str(self.MAX)
             +" N = "+str(self.N)
             +" DELAY = '"+ self.DELAY +"'\n"
            )
        """
        # make run sur toutes la machine dont le thread s'occupe avec ses paramètres
        execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS='
                                  +str(NPROCS)+' MIN='+str(self.MIN) 
                                  +' MAX='+str(self.MAX)+' N='+str(self.N)
                                  +" DELAY='"+self.DELAY
                                  +"'",hosts=nodes[self.i],connection_params={'user':'root'}).run().ok


In [21]:
# Test : trajectoire jusqu'à la ième machine raccrochée à la trajectoire initiale
def coherence(tab,i):
     return all(x == True for x in tab[0:i])

In [22]:
# Formate le tableau en string pour pouvoir etre passé en ligne de commande
def format_cmd(data):
    return (' '.join(str(e) for e in data))

In [64]:
# Formate le string en tableau d'int pour être manipulé 
def unformat_cmd(data):
    return list(map(int,data.split()))

In [24]:
# SOURCE : https://stackoverflow.com/questions/3229419/how-to-pretty-print-nested-dictionaries
def pretty(d, indent=0):
    for key, value in d.items():
        print('\t' * indent + str(key))
        if isinstance(value, type({})):
            pretty(value, indent+1)
        else:
            print('\t' * (indent+1) + str(value))

In [104]:
# SIMULATION
### Simulation simple en diminuant le nbr de machines 
def simulation(N,NPROCS,mode):
    setup() 

    # Dates: pour la gestion de fichiers
    date_precise = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
    date = date_precise.split('_')[0]
    
    # Total execution time
    total_execution_time = 0
    
    # Tableau de threads
    machines = init_tab(nodecount,None)
      
    # position trajectoire cohérente courante
    t_curr = 0
    
    # D de la trajectorie cohérente courante
    D_curr = format_cmd(init_tab(NPROCS, 0))
    
    # taille paquet d'itérations
    fragment_size = 100
    
    # Taille du dernier slot
    last_fragment_size = N % fragment_size
    
    # Nombre de fragments
    fragment_number = int((math.ceil(N/fragment_size)))
    #rounds_max = int((math.ceil(fragment_number/nodecount)))

    
    # zone d'itérations à explorer
    To_explore = collections.deque()
    
    ### Initialisation des zones à explorer D='0...0' pour chaque zone
    for i in range(fragment_number):
        To_explore.append((i * fragment_size, format_cmd(init_tab(NPROCS, 0))))
        
    print("To_explore : "+str(To_explore))
    # Round actuel
    curr_round = 0
    
    # Liste des trajectoires pour un temps t1 et un D donnée
    sim = {}
    while(True):
        curr_round = curr_round + 1
        
        # Trajectoire cohérente complète trouvé : FIN DE SIMULATION
        if(t_curr >= N) :
            print("Total time : " + str(total_execution_time))
            break;        

        print("-------------------------------------------- ROUND :  " + str(curr_round) + "   --------------------------------------------\n")
        
        # Zones restantes à explorer
        number_paths_To_explore = len(To_explore)
        
        # Soit j'ai plus de zone à explorer que de machines disponibles soit l'inverse
        for q in range(min(nodecount,number_paths_To_explore)):
            
            # On récupère la première zone à explorer
            parameters = To_explore.popleft()
            
            # MIN est le champ de gauche
            MIN = parameters[0]
            
            # MAX de la zone à explorer vaut le min + la taille du slot
            MAX = MIN + fragment_size
            
            # D est le champ de droite
            delay = parameters[1]
            
            # Paramètrage des machines
            machines[q] = ParSimer(q, MIN, MAX, N, delay)
           # print("Machine " + str(q) + " MIN :" + str(MIN) + " MAX : " + str(MAX) + " D : "+ delay)   
                
            
        # Lancement de la simulation
        for q in range(min(nodecount,number_paths_To_explore)):
            machines[q].start()
            
        # Fin de simulation
        for q in range(min(nodecount,number_paths_To_explore)) :
            machines[q].join()
            # Récupération des résultats
            execo.action.Get(hosts=nodes[q],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location= path_rslt
                     + date
                     + '/' + get_host_shortname(nodes[q]) + '_'
                     + date_precise + '.txt', connection_params={'user': 'root'
                     }).run().ok    
        # Traitement des données
        for q in range(min(nodecount,number_paths_To_explore)):
            # On récupère les paramètres des machines
            t = machines[q].get_MIN()
            D = machines[q].get_delay()
            
            # Calcul du délai
            raw_data = delayer(q,NPROCS,date_precise)
            Dprime = format_cmd(raw_data[0])
            T = raw_data[1]
            
            
            # Ajout dans ma table
            sim.setdefault(t,{}).update({D:(Dprime,curr_round,q,T)})
            MAX = t + fragment_size
            
            # Si je suis dans une zone valide
            if(MAX < N):
                # Ajout dans les zones à explorer du nouveau point de départ à exploiter
                To_explore.append((MAX,Dprime))
            #print(list(sim[t].keys()))
            #if(q not in (nodecount, number_paths_To_explore)):  
               # print("~~"+list(sim[t].keys())[len(sim[t])-1]+"~~")
                
        print("T_curr : "+ str(t_curr)+ "         D_curr : "+ str(D_curr)+ "         Dprime " + str(Dprime)+"         total execution time  "+str(total_execution_time) )
        
        # Calcul de la trajectoire cohérente
        while (t_curr != N and D_curr in sim[t_curr]):
            total_execution_time += sim[t_curr][D_curr][3]
            if (t_curr + fragment_size >= N) : total_execution_time += max(unformat_cmd(sim[t_curr][D_curr][0]))
            D_curr = sim[t_curr][D_curr][0]
            t_curr += fragment_size
            
            
        print("T_curr : "+ str(t_curr)+ "         D_curr : "+ str(D_curr)+ "         Dprime " + str(Dprime)+"         total execution time  "+str(total_execution_time) +"\n") 
        
        pretty(sim)   

In [105]:
# Nbr pcs
NPROCS = 3
    
# Nbr itérations
N = 1000

# mode de génération de l'état initial à chaque round
mode = ["FIX","RAND","RAND_INF","RAND_SUP"]

# Lancement de la simulation
simulation(N,NPROCS,mode[1])

To_explore : deque([(0, '0 0 0'), (100, '0 0 0'), (200, '0 0 0'), (300, '0 0 0'), (400, '0 0 0'), (500, '0 0 0'), (600, '0 0 0'), (700, '0 0 0'), (800, '0 0 0'), (900, '0 0 0')])
-------------------------------------------- ROUND :  1   --------------------------------------------

T_curr : 0         D_curr : 0 0 0         Dprime 9 0 15         total execution time  0
Adding :39
T_curr : 100         D_curr : 13 12 0         Dprime 9 0 15         total execution time  39

0
	0 0 0
		('13 12 0', 1, 0, 39)
200
	0 0 0
		('9 0 15', 1, 2, 42)
100
	0 0 0
		('12 0 13', 1, 1, 39)
-------------------------------------------- ROUND :  2   --------------------------------------------

T_curr : 100         D_curr : 13 12 0         Dprime 2 2 0         total execution time  39
T_curr : 100         D_curr : 13 12 0         Dprime 2 2 0         total execution time  39

0
	0 0 0
		('13 12 0', 1, 0, 39)
400
	0 0 0
		('0 2 2', 2, 1, 46)
100
	0 0 0
		('12 0 13', 1, 1, 39)
200
	0 0 0
		('9 0 15', 1, 2, 42

T_curr : 400         D_curr : 7 16 0         Dprime 6 5 0         total execution time  194
T_curr : 400         D_curr : 7 16 0         Dprime 6 5 0         total execution time  194

0
	0 0 0
		('13 12 0', 1, 0, 39)
400
	15 0 12
		('22 0 13', 10, 1, 49)
	0 0 0
		('0 2 2', 2, 1, 46)
	2 2 0
		('9 0 4', 5, 1, 46)
	14 0 9
		('25 3 0', 8, 0, 46)
700
	0 0 0
		('0 1 1', 3, 1, 47)
	22 0 20
		('15 0 22', 11, 1, 45)
	1 1 0
		('2 0 2', 6, 1, 53)
	3 9 0
		('8 11 0', 9, 0, 46)
100
	0 0 0
		('12 0 13', 1, 1, 39)
	13 12 0
		('12 15 0', 4, 1, 50)
200
	0 0 0
		('9 0 15', 1, 2, 42)
	12 0 13
		('12 0 16', 4, 2, 50)
	12 15 0
		('11 16 0', 7, 1, 52)
500
	0 0 0
		('2 2 0', 2, 2, 46)
	9 0 4
		('0 4 17', 8, 1, 45)
	0 2 2
		('13 0 11', 5, 2, 42)
	25 3 0
		('29 0 5', 10, 2, 50)
900
	0 1 1
		('0 4 3', 7, 0, 50)
	0 0 0
		('1 0 4', 4, 0, 49)
	0 5 11
		('0 11 12', 9, 2, 46)
300
	12 0 16
		('15 0 12', 7, 2, 52)
	0 0 0
		('2 2 0', 2, 0, 46)
	9 0 15
		('14 0 9', 5, 0, 49)
	11 16 0
		('7 16 0', 10, 0, 53)
600
	0 0 0


T_curr : 700         D_curr : 25 12 0         Dprime 33 0 27         total execution time  331
Adding :58
T_curr : 800         D_curr : 20 1 0         Dprime 33 0 27         total execution time  389

0
	0 0 0
		('13 12 0', 1, 0, 39)
400
	15 0 12
		('22 0 13', 10, 1, 49)
	0 0 0
		('0 2 2', 2, 1, 46)
	2 2 0
		('9 0 4', 5, 1, 46)
	7 16 0
		('12 9 0', 12, 1, 45)
	14 0 9
		('25 3 0', 8, 0, 46)
700
	35 0 6
		('30 0 4', 15, 0, 50)
	3 9 0
		('8 11 0', 9, 0, 46)
	22 0 20
		('15 0 22', 11, 1, 45)
	41 0 23
		('41 0 26', 16, 1, 49)
	0 0 0
		('0 1 1', 3, 1, 47)
	25 12 0
		('20 1 0', 17, 1, 58)
	1 1 0
		('2 0 2', 6, 1, 53)
	8 0 14
		('2 0 13', 13, 1, 45)
100
	0 0 0
		('12 0 13', 1, 1, 39)
	13 12 0
		('12 15 0', 4, 1, 50)
200
	0 0 0
		('9 0 15', 1, 2, 42)
	12 0 13
		('12 0 16', 4, 2, 50)
	12 15 0
		('11 16 0', 7, 1, 52)
500
	9 0 4
		('0 4 17', 8, 1, 45)
	0 2 2
		('13 0 11', 5, 2, 42)
	12 9 0
		('28 12 0', 14, 1, 41)
	25 3 0
		('29 0 5', 10, 2, 50)
	0 0 0
		('2 2 0', 2, 2, 46)
	22 0 13
		('22 0 8', 1

#### Publication des résultats

In [70]:
%%bash -s $experiment_name $site
git add .
git commit -m 'Automatic results update : '"$1"'---'"$2""Vérifications valeur "

[master 205d332] Automatic results update : Time_Bcast---rennes T : simulation time
 Committer: Najwa Ez Zine <nezzine@frennes>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly. Run the
following command and follow the instructions in your editor to edit
your configuration file:

    git config --global --edit

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 43 files changed, 293 insertions(+), 692 deletions(-)
 create mode 100644 SimGrid/examples/SMPI/Time_Bcast/results/2018-06-13/parasilo-5_2018-06-13_10:55.txt
 create mode 100644 SimGrid/examples/SMPI/Time_Bcast/results/2018-06-13/parasilo-5_2018-06-13_12:41.txt
 create mode 100644 SimGrid/examples/SMPI/Time_Bcast/results/2018-06-13/parasilo-5_2018-06-13_12:42.txt
 create mode 100644 SimGrid/examples/SMPI/Time_Bcast/results/2018-06-1

#### Fin d'expérience : suppression du job

In [ ]:
oardel(jobid)